In [ ]:
dnn_attack_intensity = 5

In [ ]:
from models.ae import AE
from models.dnnAE import DNN

import torch
import torch.nn.functional as F

from sklearn import metrics

import re
import os
import numpy as np
from tqdm import tqdm
from scipy.io import loadmat
import matplotlib.pyplot as plt
from collections import defaultdict
from pandas import DataFrame, concat

In [ ]:
seed = 13
torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Path to all dataset
DATASET_PATH = os.path.join(os.getcwd(), 'data')
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.path.join(os.getcwd(), 'checkpoints')

# **DNN Data Generation**

In [ ]:
Z = defaultdict(dict)
for severity in tqdm(range(5,10), desc='Reading data ... '):
    ATTACK_DATA_PATH = os.path.join(DATASET_PATH, f'I{severity}')
    for filename in os.listdir(ATTACK_DATA_PATH):
        fname = os.path.join(ATTACK_DATA_PATH, filename)
        data = loadmat(fname)
        att_type = int(re.findall(r'\d+', filename)[0])
        Z[severity][att_type] = data['Z']

NORMAL_DATA_FILENAME = 'Normal2.mat'
data = loadmat(os.path.join(DATASET_PATH, NORMAL_DATA_FILENAME))
Z[0] = data['Z']

Z = dict(sorted(Z.items()))

In [ ]:
latent_dim = 118
no_residuals = 1

## **Load Trained Models**

In [ ]:
noise_levels = [0, 5, 10, 20, 50]
aes = dict()
for level in noise_levels:
    ae = AE(latent_dim).to(device)
    ae.load_state_dict(torch.load(os.path.join(CHECKPOINT_PATH, f'ae_{level}%.pt')))
    aes[level] = ae

dnn = DNN(no_residuals=no_residuals).to(device)
dnn.load_state_dict(torch.load(os.path.join(CHECKPOINT_PATH, f'dnn_{dnn_attack_intensity}.pt')))

## **Normal Sensor Data To DNN Data**

In [ ]:
pred_normal = defaultdict()
for level in tqdm(noise_levels, desc=' Generating residuals ... '):
    ae = aes[level].eval()
    z = torch.tensor(Z[0], dtype=torch.float32).to(device)
    # AE residuals for normal data
    r = (ae(z) - z)
    pred_normal[level] = dnn(torch.stack([r], dim=1)).detach().cpu()

## **Attack Sensor Data To DNN Data**

In [ ]:
pred = defaultdict()
for level in tqdm(noise_levels, desc=' Generating residuals ... '):
    vae = vaes[level].eval()
    rnn = rnns[level].eval()
    for severity in range(5,10):
        _Z, _RES = Z[severity], RES[severity]
        R = defaultdict(list)
        for k, v in _Z.items():
            # state estimation residual
            r1 = torch.tensor(_RES[k], dtype=torch.float32)
            z = torch.tensor(v, dtype=torch.float32).to(device)
            _R = []
            for _ in range(samples):
                # VAE residuals
                z_rec = vae(z)
                r2 = (z_rec - z).detach().cpu()
                
                # rnn prediction residual
                s_t = vae.encoder(z).cpu().detach().numpy()
                s_t = series_to_supervised(s_t, n_in=rnn_window).values[:,:rnn_window * latent_dim]
                s_t = torch.tensor(s_t, dtype=torch.float32).to(device)
                z_rnn = vae.decoder(rnn(s_t))
                r3 = (z_rnn - z[rnn_window:]).detach().cpu()
                _R.append(torch.stack([r1[rnn_window:],r2[rnn_window:],r3], dim=1))
            R[k] = torch.stack(_R, dim=1)
        R = dict(sorted(R.items()))
        _pred = dnn(torch.cat(list(R.values()), dim=0).to(device))
        pred[level,severity] = torch.cat([pred_normal[level].cpu(), _pred.cpu()], dim=0).detach()

In [ ]:
y = []
no_signal = len(Z[0]) - rnn_window
for l in range(4):
    y.append(F.one_hot(l * torch.ones((no_signal), dtype=torch.long), num_classes=4))
y = torch.cat(y, dim=0)

# **DNN Evaluation**

## **Confusion Matrices**

### **Confusion Matrix: Noise Level 0%**

In [ ]:
severity = 5
level = 0
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 6
level = 0
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 7
level = 0
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 8
level = 0
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 9
level = 0
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

### **Confusion Matrix: Noise Level 5%**

In [ ]:
severity = 5
level = 5
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 6
level = 5
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 7
level = 5
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 8
level = 5
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 9
level = 5
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

### **Confusion Matrix: Noise Level 10%**

In [ ]:
severity = 5
level = 10
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 6
level = 10
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 7
level = 10
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 8
level = 10
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 9
level = 10
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

### **Confusion Matrix: Noise Level 20%**

In [ ]:
severity = 5
level = 20
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 6
level = 20
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()


In [ ]:
severity = 7
level = 20
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 8
level = 20
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 9
level = 20
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

### **Confusion Matrix: Noise Level 50%**

In [ ]:
severity = 5
level = 50
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 6
level = 50
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 7
level = 50
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 8
level = 50
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

In [ ]:
severity = 9
level = 50
pp = torch.argmax(pred[level,severity], dim=1).numpy()
yy = torch.argmax(y, dim=1).numpy()
# report
print("Classification Report:\n")
print(metrics.classification_report(yy, pp, zero_division=0))
# confusion matrix
cm = metrics.confusion_matrix(yy,pp, labels=[0,1,2,3])
print("Confusion Matrix:")
metrics.ConfusionMatrixDisplay(cm, display_labels=[0,1,2,3]).plot()
plt.show()
# roc curve 0 vs. rest
chance_level_line_kw = {
            "label": "Chance level (AUC = 0.5)",
            "color": "k",
            "linestyle": "--",
        }
yy[yy>0] = 1
pp[pp>0] = 1
metrics.RocCurveDisplay.from_predictions(
    yy,
    pp,
    name="No Attack vs the rest",
    color="darkorange"
)
plt.plot((0, 1), (0, 1), **chance_level_line_kw)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves: Attack Detection")
plt.legend()
plt.show()

## **Quantiles**

In [ ]:
q = torch.tensor([0.025, 0.975])
x_ax = ['0%', '%5', '%10', '%20', '50%']

### **Quantiles: Attack Severity 5**

In [ ]:
severity = 5
mean, low, up = defaultdict(), defaultdict(), defaultdict()
_min = defaultdict()
for k, v in tqdm(pred.items(),desc=' Generating quantiles ... '):
    if k[1] == severity:
        for _type in range(4):
            _pred = v[_type * no_signal:(_type + 1) * no_signal]
            _min_val, _min_id = _pred.min(dim=0)
            _min[k[0],_type] = {_min_id[_type].item(): _min_val[_type].item()}
            mean[k[0],_type] = _pred.mean(dim=0).numpy()[_type]
            _low, _up = torch.quantile(_pred, q, dim=0).numpy()
            low[k[0], _type], up[k[0], _type] = _low[_type], _up[_type]

In [ ]:
_min

In [ ]:
x = np.arange(len(noise_levels))
fig, ax = plt.subplots(1, 4, figsize=(25, 4))
for _type in range(4):
    _m = [mean[l, _type] for l in noise_levels]
    _l = [low[l, _type] for l in noise_levels]
    _u = [up[l, _type] for l in noise_levels]
    ax[_type].plot(x, _l, label='Lower', lw=3, color='g')
    ax[_type].plot(x, _u, label='Upper', lw=3, color='b')
    ax[_type].plot(x, _m, label='Mean', lw=3, color='r')
    ax[_type].fill_between(x, _l, _u, color='r', alpha=0.2)
    ax[_type].set(xticks=x, xticklabels=x_ax)
    if _type == 0:
        ax[_type].title.set_text('Normal Data')
    else:
        ax[_type].title.set_text(f'Attack Data {_type}')
    ax[_type].legend()
plt.show()

### **Quantiles: Attack Severity 6**

In [ ]:
severity = 6
mean, low, up = defaultdict(), defaultdict(), defaultdict()
for k, v in tqdm(pred.items(),desc=' Generating quantiles ... '):
    if k[1] == severity:
        for _type in range(4):
            _pred = v[_type * no_signal:(_type + 1) * no_signal]
            mean[k[0],_type] = _pred.mean(dim=0).numpy()[_type]
            _low, _up = torch.quantile(_pred, q, dim=0).numpy()
            low[k[0], _type], up[k[0], _type] = _low[_type], _up[_type]

In [ ]:
x = np.arange(len(noise_levels))
fig, ax = plt.subplots(1, 4, figsize=(25, 4))
for _type in range(4):
    _m = [mean[l, _type] for l in noise_levels]
    _l = [low[l, _type] for l in noise_levels]
    _u = [up[l, _type] for l in noise_levels]
    ax[_type].plot(x, _l, label='Lower', lw=3, color='g')
    ax[_type].plot(x, _u, label='Upper', lw=3, color='b')
    ax[_type].plot(x, _m, label='Mean', lw=3, color='r')
    ax[_type].fill_between(x, _l, _u, color='r', alpha=0.2)
    ax[_type].set(xticks=x, xticklabels=x_ax)
    if _type == 0:
        ax[_type].title.set_text('Normal Data')
    else:
        ax[_type].title.set_text(f'Attack Data {_type}')
    ax[_type].legend()
plt.show()

### **Quantiles: Attack Severity 7**

In [ ]:
severity = 7
mean, low, up = defaultdict(), defaultdict(), defaultdict()
for k, v in tqdm(pred.items(),desc=' Generating quantiles ... '):
    if k[1] == severity:
        for _type in range(4):
            _pred = v[_type * no_signal:(_type + 1) * no_signal]
            mean[k[0],_type] = _pred.mean(dim=0).numpy()[_type]
            _low, _up = torch.quantile(_pred, q, dim=0).numpy()
            low[k[0], _type], up[k[0], _type] = _low[_type], _up[_type]

In [ ]:
x = np.arange(len(noise_levels))
fig, ax = plt.subplots(1, 4, figsize=(25, 4))
for _type in range(4):
    _m = [mean[l, _type] for l in noise_levels]
    _l = [low[l, _type] for l in noise_levels]
    _u = [up[l, _type] for l in noise_levels]
    ax[_type].plot(x, _l, label='Lower', lw=3, color='g')
    ax[_type].plot(x, _u, label='Upper', lw=3, color='b')
    ax[_type].plot(x, _m, label='Mean', lw=3, color='r')
    ax[_type].fill_between(x, _l, _u, color='r', alpha=0.2)
    ax[_type].set(xticks=x, xticklabels=x_ax)
    if _type == 0:
        ax[_type].title.set_text('Normal Data')
    else:
        ax[_type].title.set_text(f'Attack Data {_type}')
    ax[_type].legend()
plt.show()

### **Quantiles: Attack Severity 8**

In [ ]:
severity = 8
mean, low, up = defaultdict(), defaultdict(), defaultdict()
for k, v in tqdm(pred.items(),desc=' Generating quantiles ... '):
    if k[1] == severity:
        for _type in range(4):
            _pred = v[_type * no_signal:(_type + 1) * no_signal]
            mean[k[0],_type] = _pred.mean(dim=0).numpy()[_type]
            _low, _up = torch.quantile(_pred, q, dim=0).numpy()
            low[k[0], _type], up[k[0], _type] = _low[_type], _up[_type]

In [ ]:
x = np.arange(len(noise_levels))
fig, ax = plt.subplots(1, 4, figsize=(25, 4))
for _type in range(4):
    _m = [mean[l, _type] for l in noise_levels]
    _l = [low[l, _type] for l in noise_levels]
    _u = [up[l, _type] for l in noise_levels]
    ax[_type].plot(x, _l, label='Lower', lw=3, color='g')
    ax[_type].plot(x, _u, label='Upper', lw=3, color='b')
    ax[_type].plot(x, _m, label='Mean', lw=3, color='r')
    ax[_type].fill_between(x, _l, _u, color='r', alpha=0.2)
    ax[_type].set(xticks=x, xticklabels=x_ax)
    if _type == 0:
        ax[_type].title.set_text('Normal Data')
    else:
        ax[_type].title.set_text(f'Attack Data {_type}')
    ax[_type].legend()
plt.show()

### **Quantiles: Attack Severity 9**

In [ ]:
severity = 9
mean, low, up = defaultdict(), defaultdict(), defaultdict()
for k, v in tqdm(pred.items(),desc=' Generating quantiles ... '):
    if k[1] == severity:
        for _type in range(4):
            _pred = v[_type * no_signal:(_type + 1) * no_signal]
            mean[k[0],_type] = _pred.mean(dim=0).numpy()[_type]
            _low, _up = torch.quantile(_pred, q, dim=0).numpy()
            low[k[0], _type], up[k[0], _type] = _low[_type], _up[_type]

In [ ]:
x = np.arange(len(noise_levels))
fig, ax = plt.subplots(1, 4, figsize=(25, 4))
for _type in range(4):
    _m = [mean[l, _type] for l in noise_levels]
    _l = [low[l, _type] for l in noise_levels]
    _u = [up[l, _type] for l in noise_levels]
    ax[_type].plot(x, _l, label='Lower', lw=3, color='g')
    ax[_type].plot(x, _u, label='Upper', lw=3, color='b')
    ax[_type].plot(x, _m, label='Mean', lw=3, color='r')
    ax[_type].fill_between(x, _l, _u, color='r', alpha=0.2)
    ax[_type].set(xticks=x, xticklabels=x_ax)
    if _type == 0:
        ax[_type].title.set_text('Normal Data')
    else:
        ax[_type].title.set_text(f'Attack Data {_type}')
    ax[_type].legend()
plt.show()